In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel("Sample Data.xlsx")

# EDA

In [3]:
df.shape

(193927, 12)

In [4]:
df.sample(5)

,Period,Năm,Tháng,BU,GEO,Province2,Product,Brand,Brand2,Price Class,Qty,Revenue
76266,2019-02,2019,2,TGDĐ,Miền Bắc 2,Quang Tri,MTXT,Asus,Asus,10M - 12M,2,19436364
22502,2017-12,2017,12,Trần Anh,Miền Bắc 1,Lang Son,MTXT,Acer,Acer,8M - 10M,1,8172727
33265,2018-03,2018,3,TGDĐ,Miền Trung,Quang Ngai,MTXT,Asus,Asus,>16M,2,25963636
112032,2019-11,2019,11,TGDĐ,Miền Bắc 2,Quang Tri,MTXT,HP,HP,14M - 16M,2,27454545
42428,2018-06,2018,6,ĐMX,Miền Trung,Phu Yen,MTXT,Asus,Asus,>16M,4,65327887


In [5]:
df.dtypes

Period         object
Năm             int64
Tháng           int64
BU             object
GEO            object
Province2      object
Product        object
Brand          object
Brand2         object
Price Class    object
Qty             int64
Revenue         int64
dtype: object

### Change type of Period to datime

In [6]:
df.Period = pd.to_datetime(df.Period, format="%Y-%m")

# EDA (cont)

In [7]:
df.select_dtypes(include=['int64','datetime']).agg([min,max])

,Period,Năm,Tháng,Qty,Revenue
min,2017-07-01,2017,1,-3,-76677273
max,2021-04-01,2021,12,438,11563590591


In [8]:
start_time, end_time = df.Period.agg([min,max]).values

In [9]:
df.BU.value_counts()

FRT         71534
TGDĐ        60318
ĐMX         58901
Trần Anh     3151
x              23
Name: BU, dtype: int64

In [10]:
df.GEO.value_counts()

Tây Nam Bộ     42112
Miền Trung     41170
Miền Bắc 1     41105
Miền Bắc 2     36406
Đông Nam Bộ    22715
HNI             4990
HCM             4954
Du an            383
Du an - DA        60
x                 32
Name: GEO, dtype: int64

In [11]:
df.Product.value_counts()

MTXT    193927
Name: Product, dtype: int64

### Drop rows when value = 'x'

In [12]:
df = df[df.GEO != 'x']

In [13]:
df.Province2.value_counts()

Ho Chi Minh    5207
Ha Noi         5180
Dong Nai       4295
Binh Duong     4243
Da Nang        4057
               ... 
Cao Bang       1755
Dien Bien      1747
Lai chau       1439
Bac Kan        1335
Lai Chau         12
Name: Province2, Length: 64, dtype: int64

In [14]:
df.Province2 = df.Province2.replace('Lai chau', 'Lai Chau')

# Preprocessing

### Genarate date dimension

In [15]:
dim_date = pd.DataFrame({"Date": pd.date_range(start=start_time,end= end_time)})
dim_date['DATE_KEY'] = dim_date.Date.dt.strftime('%Y%m%d')
dim_date['Month'] = dim_date.Date.dt.strftime('%b')
dim_date['Year'] = dim_date.Date.dt.year
dim_date['Weekday']  = dim_date.Date.dt.strftime('%A')
dim_date['Weekday No'] = np.add(dim_date.Date.dt.dayofweek,1)
dim_date['Day of Month'] = dim_date.Date.dt.day
dim_date['Month of Year'] = dim_date.Date.astype('datetime64[M]').dt.strftime('%A, %B %#d, %Y')
dim_date['Quarter'] = dim_date.Date.dt.quarter
dim_date['Week of Month'] = (dim_date.Date.dt.day - 1) // 7 + 1
dim_date['Week of Year'] = dim_date.Date.dt.isocalendar().week
dim_date['Month No'] = dim_date.Date.dt.month
dim_date['Date'] = dim_date.Date.dt.strftime('%d/%m/%Y %I:%M:%S %p')

In [16]:
#dim_date.to_excel('dim_date.xlsx',index=False)

### Create dimension table

In [17]:
dim_distributor = df[['BU','GEO','Province2']].drop_duplicates().reset_index(drop=True)
#dim_distributor.columns = ['BU', 'Location Level 1', 'Location Level 2']
dim_distributor['DISTRIBUTOR_KEY'] = pd.Series(np.arange(len(dim_distributor)))

In [18]:
dim_product = df[['Product','Brand2', 'Brand', 'Price Class']].drop_duplicates().reset_index(drop=True)
#dim_product.columns = ['Product', 'Brand Level 1', 'Brand Level 2', 'Price Class']
dim_product['PRODUCT_KEY'] = pd.Series(np.arange(len(dim_product)))

### Create fact table

In [19]:
fact_table = df.copy()
fact_table['DATE_KEY'] = df.Period.dt.strftime('%Y%m%d')
fact_table = fact_table.drop(columns= ['Period', 'Năm', 'Tháng'])

In [20]:
fact_table = fact_table.merge(dim_distributor).drop(columns=['BU','GEO','Province2'])
fact_table = fact_table.merge(dim_product).drop(columns=['Product','Brand2', 'Brand', 'Price Class'])

In [21]:
fact_table = fact_table.rename(columns= {'Qty': 'QUANTITY', 'Revenue': 'REVENUE'})

### Add order columns for dimension distributor

In [22]:
bu_order = df.BU.value_counts().rank(method='max').astype('int').to_dict()
dim_distributor['BU_ORDER'] = dim_distributor.BU.map(bu_order)

In [23]:
loc_order= {'Miền Bắc 1': 0, 'Miền Bắc 2': 1, 'HNI': 2, 'Miền Trung': 3, 
            'Đông Nam Bộ': 4, 'HCM': 5, 'Tây Nam Bộ': 6, 'Du an': 7, 'Du an - DA': 8}
dim_distributor['LOCATION_ORDER'] = dim_distributor.GEO.map(loc_order)

In [24]:
dim_distributor = dim_distributor.rename(columns= {'GEO': 'Location Level 1', 'Province2': 'Local Level 2'})

### Add order columns for dimension product

In [25]:
l = dim_product['Price Class'].unique()
l2 = [int(re.search(r'\d+', e).group()) for e in l]
l3 = [len(e) for e in l]

In [26]:
price_class_order = pd.DataFrame(np.array([l,l2,l3]).T).sort_values(by=[1,2]).reset_index(drop=True)
price_class_order = price_class_order.reset_index().set_index(0)
price_class_order= price_class_order['index'].to_dict()

In [27]:
dim_product['PRICE_CLASS_ORDER'] = dim_product['Price Class'].map(price_class_order)

In [28]:
dim_product = dim_product.rename(columns= {'Brand2': 'Brand Level 1', 'Brand': 'Brand Level 2'})

In [30]:
dim_date.to_excel('data/Date.xlsx', index=False)
dim_product.to_excel('data/Product.xlsx', index= False)
dim_distributor.to_excel('data/Distributor.xlsx', index= False)
fact_table.to_excel('data/Sales.xlsx', index= False)

In [37]:
dim_distributor.BU_ORDER.value_counts()

4    67
3    63
2    63
1    20
Name: BU_ORDER, dtype: int64